In [80]:
# Package imports
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import pandas as pd
import numpy as np
from decimal import Decimal


In [61]:
# Use pandas to read the CSV file as a dataframe
df = pd.read_csv("moons400.csv")
# The y values are those labelled 'Class': extract their values
y = df['Class'].values

# The x values are all other columns
del df['Class']    # drop the 'Class' column from the dataframe
X = df.as_matrix() # convert the remaining columns to a numpy array
# Some examples of working with the data, to look at rows/columns
print ("len(X):", len(X))            # outer array: one per sample
print ("len(X[0]):", len(X[0]))      # each inner array is the attributes of one sample
print ("len(X[:,0]):", len(X[:,0]))  # select column 0 from array

# np.shape returns all dimensions of the array
(nsamples, nattribs) = np.shape(X)
print ("X: nsamples =", nsamples, ", nattribs =", nattribs)

len(X): 400
len(X[0]): 2
len(X[:,0]): 400
X: nsamples = 400 , nattribs = 2


In [62]:
datasubset_x = X[0:1]
(dsamples, dattribs) = np.shape(datasubset_x)
print("datasubset: dsamples =", dsamples, ", dattribs =", dattribs)
print(datasubset_x)

datasubset: dsamples = 1 , dattribs = 2
[[ 2.07106946  0.41152931]]


In [73]:
inputs = datasubset_x
inputs[0,0] = 0.05
inputs[0,1] = 0.1
print(inputs)

[[ 0.05  0.1 ]]


In [64]:
# target = y[0:1]
target = 0.01
print(target)

0.01


In [91]:
# forward propagation for 1 record
datasubset_x
# result = [[0]],[0]]

W11_1 = 0.15
W12_1 = 0.2
W21_1 = 0.25
W22_1 = 0.3

b1_1 = 0.35000


W11_2 = 0.4
W12_2 = 0.45
b_2 = 0.6

# print(datasubset_x[0,1])



a1_2 = datasubset_x[0,0]*W11_1 + datasubset_x[0,1]*W12_1 + b1_1
a2_2 = datasubset_x[0,0]*W21_1 + datasubset_x[0,1]*W22_1 + b1_1
print( a1_2)
print (a2_2)

a1_2 = 1/(1+np.exp(-a1_2))
a2_2 = 1/(1+np.exp(-a2_2))
print(a1_2)
print(a2_2)

output = 1/(1+np.exp(-(W11_2*a1_2 + W12_2*a2_2 + b_2)))

print( output)

[[0.15, 0.2], [0.25, 0.3]]
[[ 0.0075  0.02  ]
 [ 0.0125  0.03  ]]
[ 0.02  0.05]
0.3775
0.3925
0.593269992107
0.59688437826
0.751365069552


In [103]:
# do the above using matrix multiplication
Weights1 = [[0.15000,0.20000],[0.25000,0.30000]]
bias_1 = 0.35
print (Weights1)

Weights2 = [0.4, 0.45]
bias_2 = 0.6

# trying to get it working using matrix multiplication and addition

# hidden = np.array(Weights1*inputs)
hidden = Weights1*inputs
print(hidden)


# when I sum the columns its rounding the value so losing accuracy
hidden = sum(hidden)+bias_1  #  need to get 4 decimal places here
print(hidden)

activation = 1/(1+np.exp(-hidden))
print(activation)

output = (activation*Weights2)
output = 1/(1+np.exp(-(sum(output)+bias_2)))
print(output)


[[0.15, 0.2], [0.25, 0.3]]
[[ 0.0075  0.02  ]
 [ 0.0125  0.03  ]]
[ 0.37  0.4 ]
[ 0.59145898  0.59868766]
0.751381335486


In [66]:
# get the error 
error_o = (0.5*(target - output)**2)
print(error_o)

0.274811083176


In [67]:
# calculate derivative of error at output layer
deriv_wrt_out = -(target - output)
derivout_wrt_net = output*(1-output)
deriv_wa11 = deriv_wrt_out*derivout_wrt_net*a1_2
deriv_wa12 = deriv_wrt_out*derivout_wrt_net*a2_2
print(deriv_wrt_out)
print(derivout_wrt_net)
print(deriv_wa11)
print(deriv_wa12)

0.741365069552
0.186815601809
0.0821670405642
0.0826676278475


In [68]:
# calculate derivative of error at hidden layer

deriv_out1_wrt_h1 = deriv_wrt_out*derivout_wrt_net*W11_2
deriv_outh1_wrt_neth1 = a1_2*(1-a1_2)
deriv_net_wrt_w1 = datasubset_x[0,0]

derivtot_wrt_w1 = deriv_out1_wrt_h1*deriv_outh1_wrt_neth1*deriv_net_wrt_w1

print(derivtot_wrt_w1)

deriv_out1_wrt_h1 = deriv_wrt_out*derivout_wrt_net*W11_2
deriv_outh1_wrt_neth1 = a1_2*(1-a1_2)
deriv_net_wrt_w2 = datasubset_x[0,1]

derivtot_wrt_w2 = deriv_out1_wrt_h1*deriv_outh1_wrt_neth1*deriv_net_wrt_w2

print(derivtot_wrt_w2)

deriv_out2_wrt_h2 = deriv_wrt_out*derivout_wrt_net*W12_2
deriv_outh2_wrt_neth2 = a2_2*(1-a2_2)
deriv_net_wrt_w3 = datasubset_x[0,0]

derivtot_wrt_w3 = deriv_out2_wrt_h2*deriv_outh2_wrt_neth2*deriv_net_wrt_w3

print(derivtot_wrt_w3)

deriv_out2_wrt_h2 = deriv_wrt_out*derivout_wrt_net*W12_2
deriv_outh2_wrt_neth2 = a2_2*(1-a2_2)
deriv_net_wrt_w4 = datasubset_x[0,1]

derivtot_wrt_w4 = deriv_out2_wrt_h2*deriv_outh2_wrt_neth2*deriv_net_wrt_w4

print(derivtot_wrt_w4)


0.000668396021144
0.00133679204229
0.000749803774445
0.00149960754889


In [69]:
# get the the adjusted weight of the output node
# using the derivative of the error

alpha = 0.5 # learning rate

W11_2 = W11_2 - (alpha*deriv_wa11)
W12_2 = W12_2 - (alpha*deriv_wa12)

print (W11_2)
print (W12_2)

W11_1 = W11_1 - (alpha*derivtot_wrt_w1)
print (W11_1)

W12_1 = W12_1 - (alpha*derivtot_wrt_w2)
print (W12_1)

W21_1 = W21_1 - (alpha*derivtot_wrt_w3)
print (W21_1)

W22_1 = W22_1 - (alpha*derivtot_wrt_w4)
print (W22_1)

0.358916479718
0.408666186076
0.149665801989
0.199331603979
0.249625098113
0.299250196226
